In [1]:
import os
import sys
sys.path.append(os.path.abspath('') + '/..')

In [2]:
import pandas as pd 
import re
import numpy as np

from data_utils import getDF
from dataset_evaluation_utils import * 

from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('whitegrid')

# import plotly.offline as py
# pd.options.plotting.backend = "plotly"
# py.init_notebook_mode() # graphs charts inline (IPython).

a4_dims = (11.7, 8.27)

# extract file

In [3]:
# import tarfile 
  
# # open file 
# file = tarfile.open('../datasets/lastfm-dataset-1K.tar.gz') 
  
# # extracting file 
# file.extractall('../datasets/') 
  
# file.close() 

# paths

In [4]:
dataset_name = 'LastFM_1k'
dump_foldername ='lastfm_1k_dump/'

# rule: what/which_data_set/sample_version/what/
images_path, output_path, heatmaps_path, diversity_graphpath, diversity_filepath = get_folderpaths(dump_foldername)

# read file and save rows

In [20]:
names=['user_id', 'timestamp', 'musicbrainz-artist-id', 'artist-name', 'musicbrainz-track-id', 'track-name', 'nextline_char']
n_lines = 19150868 # from readme file
k = 1000
step = 100*k
perc = step/n_lines
# step = int(n_lines*perc)


In [ ]:
# there is no repetition of lines from earlier files within the most recent files
# it reads a bigger chunk every time

# t = 0
# all_rows = np.empty((0, len(names)), dtype=object)

# file = open('../datasets/lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv', 'r')
# for s in range(step,  n_lines+step, step):
#     rows = np.empty((0, len(names)), dtype=object)
    
#     for i in range(s):
#         line = file.readline()
#         # line_array = line.split('\t|\n')
#         line_array = re.split('\t|\n', line) # leave the last '\n' column to make sure the file is read properly
#         if (not line) | (len(line_array)!=len(names)):
#             print(line)
#             print(line_array)
#             break
#         rows = np.vstack([rows, line_array]) 

#     print(s, s/n_lines)
#     joblib.dump(rows, output_path+dataset_name+'_rows_'+format(perc, '.5f')+'_next'+format((s/k), '.0f')+'k.joblib')
#     t += s
#     all_rows = np.vstack([all_rows, rows])
#     print(all_rows)
# joblib.dump(all_rows, output_path+dataset_name+'_all_rows_perc'+format(perc, '.5f')+'_next'+format((t/k), '.0f')+'k.joblib'

In [25]:
# i stopped reading the file
print('all files written are '+ str((100*k + 200*k + 300*k + 400*k + 500*k)/n_lines) + ' of the original file')

all files written are 0.078325431515689 of the original file


In [30]:
# t = 0
# all_rows = np.empty((0, len(names)), dtype=object)
# for s in range(step,  500*k+step, step):
#     t += s
#     all_rows = np.vstack([all_rows, 
#                           joblib.load(output_path+dataset_name+'_rows_perc'+format(perc, '.5f')+'_next'+format((s/k), '.0f')+'k.joblib')])
# #     # print(all_rows)
# joblib.dump(all_rows, output_path+dataset_name+'_all_rows_perc'+format(perc, '.5f')+'_next'+format((t/k), '.0f')+'k.joblib')

['output/lastfm_1k_dump/LastFM_1k_all_rows_perc0.00522_next1500k.joblib']

# original dataframe - the first 1500k lines

In [75]:
orig_df = pd.DataFrame(joblib.load(output_path+dataset_name+'_all_rows_perc'+format(perc, '.5f')+'_next1500k.joblib')
                       , columns=names)
orig_df

,user_id,timestamp,musicbrainz-artist-id,artist-name,musicbrainz-track-id,track-name,nextline_char
0,user_000001,2009-05-04T23:08:57Z,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Deep Dish,,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007,
1,user_000001,2009-05-04T13:54:10Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,,Composition 0919 (Live_2009_4_15),
2,user_000001,2009-05-04T13:52:04Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,,Mc2 (Live_2009_4_15),
3,user_000001,2009-05-04T13:42:52Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,,Hibari (Live_2009_4_15),
4,user_000001,2009-05-04T13:42:11Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,,Mc1 (Live_2009_4_15),
...,...,...,...,...,...,...,...
1499995,user_000070,2008-07-16T10:28:04Z,5b11f4ce-a62d-471e-81fc-a69a8278c7da,Nirvana,656cfcb3-5870-4db8-8510-6cb598b47aad,Dumb,
1499996,user_000070,2008-07-14T20:20:36Z,39ab1aed-75e0-4140-bd47-540276886b60,Oasis,bc61d0bf-9b61-43c0-85ce-265b6172efda,Falling Down (Chemical Brothers Remix),
1499997,user_000070,2008-07-14T17:45:16Z,9de8f66e-3cd1-4f11-8328-38200f0612b0,Doves,c83dafb9-3bad-40a0-a682-3647950cbcc2,Sky Starts Falling,
1499998,user_000070,2008-07-14T17:39:49Z,9de8f66e-3cd1-4f11-8328-38200f0612b0,Doves,0b162982-de61-4535-89fd-e1ba5efc4bfb,Walk In Fire,


# timestamp

In [141]:
_ = orig_df.set_index('timestamp')
_.sort_index(inplace=True)
start = _.iloc[0].name
print(start)
end = _.iloc[-1].name
print(end)
(pd.Timestamp(end) - pd.Timestamp(start)).days/365

2005-02-14T11:21:05Z
2009-06-19T17:12:30Z


4.345205479452055

# NA values

In [76]:
orig_df[orig_df==''] = None
orig_df

,user_id,timestamp,musicbrainz-artist-id,artist-name,musicbrainz-track-id,track-name,nextline_char
0,user_000001,2009-05-04T23:08:57Z,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Deep Dish,None,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007,None
1,user_000001,2009-05-04T13:54:10Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,None,Composition 0919 (Live_2009_4_15),None
2,user_000001,2009-05-04T13:52:04Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,None,Mc2 (Live_2009_4_15),None
3,user_000001,2009-05-04T13:42:52Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,None,Hibari (Live_2009_4_15),None
4,user_000001,2009-05-04T13:42:11Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,None,Mc1 (Live_2009_4_15),None
...,...,...,...,...,...,...,...
1499995,user_000070,2008-07-16T10:28:04Z,5b11f4ce-a62d-471e-81fc-a69a8278c7da,Nirvana,656cfcb3-5870-4db8-8510-6cb598b47aad,Dumb,None
1499996,user_000070,2008-07-14T20:20:36Z,39ab1aed-75e0-4140-bd47-540276886b60,Oasis,bc61d0bf-9b61-43c0-85ce-265b6172efda,Falling Down (Chemical Brothers Remix),None
1499997,user_000070,2008-07-14T17:45:16Z,9de8f66e-3cd1-4f11-8328-38200f0612b0,Doves,c83dafb9-3bad-40a0-a682-3647950cbcc2,Sky Starts Falling,None
1499998,user_000070,2008-07-14T17:39:49Z,9de8f66e-3cd1-4f11-8328-38200f0612b0,Doves,0b162982-de61-4535-89fd-e1ba5efc4bfb,Walk In Fire,None


In [77]:
orig_df.isna().sum()

user_id                        0
timestamp                      0
musicbrainz-artist-id      49902
artist-name                    0
musicbrainz-track-id      193891
track-name                     0
nextline_char            1500000
dtype: int64

In [78]:
orig_df.drop(columns=['nextline_char'], inplace=True)

## track-name

In [79]:
orig_df['track-name'].str.contains('[U-u]ntitled', regex=True).value_counts()

False    1499020
True         980
Name: track-name, dtype: int64

In [86]:
untitled_filter = (orig_df['track-name'].str.contains('[U-u]ntitled', regex=True))
orig_df.loc[untitled_filter & (orig_df['track-name']!='Untitled'), ['track-name']].value_counts()

track-name                      
[Untitled]                          268
Untitled 1                           62
[Untitled Track]                     40
Sigur 4 (Untitled)                   23
Untitled (Velapene Screen Remix)     23
                                   ... 
Untitled 4 ("Njósnavélin")            1
Untitled Duet No. 3                   1
Untitled Finale                       1
Untitled Gabber Song                  1
"Untitled" (Meditation)               1
Length: 104, dtype: int64

In [87]:
orig_df.loc[untitled_filter & (orig_df['track-name']!='Untitled'), 'track-name'].unique()

array(['[Untitled]', 'Untitled 1', '[Untitled Track 5]',
       '[Untitled Track 4]', '[Untitled Track 3]', '[Untitled Track 2]',
       '[Untitled Track 1]', 'Forever Untitled',
       '[Untitled Instrumental]', 'A - Untitled [Akufen Rmx]',
       'Untitled Head', "Untitled 'Akufen Rmx", 'Untitled - Side B',
       'Untitled - Side A',
       'Untitled - Slacker Jack - Divisivle - Muff Diver',
       'B1 - Untitled', 'A1 - Untitled', 'A2. Untitled 5',
       'Untitled Loop', '(Untitled Loop)', 'Untitled B', 'Untitled A',
       'Untitled #1', '(B1) Untitled', '(A1) Untitled',
       'At The End Of It All / [Untitled]', 'Untitled (Bonus Track)',
       'Smells Like Teen Spirit (Untitled Remix)',
       'Untitled (Velapene Screen Remix)', '(Untitled)',
       'Untitled Melody', 'Untitled - (The Police Vs. Klaxons)',
       '[Untitled] (Warren Defever Remix)',
       '[Untitled] (Feat. Burzootie) (Twisted Science Remix)',
       '[Untitled] (Springheel Jack Remix)',
       '[Untitled] (S

In [84]:
# na_filter = orig_df['track-name'].str.contains('[U-u]ntitled', regex=True)
# orig_df.loc[na_filter, 'track-name'] = 'Untitled'

In [85]:
# orig_df['track-name'].value_counts()

# since there is NA ids, can i pair track-artist and create an id?
* intuitively, different songs from different artists can have the same name 
* if they are from the same artist, are they the only one they have, or does an artist have different songs with the same name?

In [88]:
orig_df.loc[untitled_filter, ['track-name', 'artist-name']].value_counts()

track-name               artist-name                    
Untitled 1               Keane                              58
Untitled                 Interpol                           48
                         Love Is All                        32
                         A Hundred Birds                    29
[Untitled Track]         Stephen Lynch                      28
                                                            ..
Untitled Lullaby         Mark Lanegan                        1
Untitled Loop            White Dolemite                      1
Untitled Ii              Nightwish                           1
Untitled I               Nightwish                           1
"Untitled" (Meditation)  Mf Doom [Special Herbs 9 And 0]     1
Length: 255, dtype: int64

In [95]:
orig_df.loc[untitled_filter, ['track-name', 'artist-name', 'musicbrainz-track-id']].value_counts()

track-name                         artist-name             musicbrainz-track-id                
Untitled 1                         Keane                   9fd1fbe7-f521-4062-99fe-feaadca82c86    58
Untitled                           Interpol                5025349a-bd35-42c9-9742-ea682ddc50b9    48
Untitled (Velapene Screen Remix)   Interpol                d844f901-0760-484d-b5e8-6914e568d174    23
Sigur 4 (Untitled)                 Sigur Rós               87a807fb-49fc-4a23-9ba0-86651e029ea5    23
[Untitled]                         The Frosted Ambassador  91212a88-714b-4f54-908b-8289d7f70b52    20
                                                                                                   ..
                                   Les Savy Fav            98d33000-26ca-4729-b8b9-f0149cf2ef24     1
                                   London After Midnight   73765ce3-e3c2-4dc6-9e71-3973b36c4c3d     1
                                   Luke Vibert             6bda03e2-ade9-493a-986a-88903

In [97]:
print('is there a "Untiled" track - artist pair with more than 1 id?\nnumber of ids per pair:')
orig_df.loc[untitled_filter, ['track-name', 'artist-name', 'musicbrainz-track-id']].drop_duplicates().value_counts().value_counts()

is there a "Untiled" track - artist pair with more than 1 id?
number of ids per pair:


1    135
dtype: int64

No, there isnt. It means that even tho the track is named untitled, it was on purpose and it isnt a NA value.

In [103]:
print('is there a track - artist pair with more than 1 id?\nnumber of ids per pair:')
orig_df.loc[:, ['track-name', 'artist-name', 'musicbrainz-track-id']].drop_duplicates().value_counts().value_counts()

is there a track - artist pair with more than 1 id?
number of ids per pair:


1    178979
dtype: int64

In [104]:
print('how many track-artist have different ids?')
orig_df.loc[:, ['track-name', 'artist-name', 'musicbrainz-track-id']]\
                    .drop_duplicates()[['track-name', 'artist-name']]\
                                            .value_counts()

how many track-artist have different ids?


track-name                         artist-name             
Who Wants To Live Forever          Dune                        3
Take Me Back To Your House         Basement Jaxx               3
Another Brick In The Wall, Part 2  Pink Floyd                  3
So Much Trouble In The World       Bob Marley & The Wailers    3
Over The Hills And Far Away        Led Zeppelin                3
                                                              ..
Red Is The New Black               Funeral For A Friend        1
                                   Avenged Sevenfold           1
Red Is A Slow Colour               The Gathering               1
Red Intro                          Fugees                      1
! (Foreword)                       Pain Of Salvation           1
Length: 228465, dtype: int64

hmmm... :I


In [105]:
orig_df.loc[(orig_df['track-name']=='Who Wants To Live Forever') & (orig_df['artist-name']=='Dune'), :]

,user_id,timestamp,musicbrainz-artist-id,artist-name,musicbrainz-track-id,track-name
996279,user_000041,2007-08-27T17:17:48Z,e497aa76-e7de-4845-a4b7-e27bb9fc9967,Dune,373bc2de-e285-4e52-a521-074643164898,Who Wants To Live Forever
999591,user_000041,2006-10-02T14:35:49Z,e497aa76-e7de-4845-a4b7-e27bb9fc9967,Dune,7fcde88f-183d-4a5f-afd5-5ef24a8a119b,Who Wants To Live Forever
1001101,user_000041,2006-03-02T18:44:16Z,e497aa76-e7de-4845-a4b7-e27bb9fc9967,Dune,0e356e50-b6d5-4e0c-ba1d-58d5a9c81dfc,Who Wants To Live Forever


oh the timetsamp is different! :D

In [108]:
orig_df[['artist-name','track-name']].groupby(['track-name']).count()

,artist-name
track-name,
! (Foreword),8
! / Sn10.1.Ai 0711,1
! / Sn2.Ai 0853,1
!@*$%#,3
!Fire In The Hole!,2
...,...
Ｒｙｄｅｅｎ,1
Ｔouch - A Mountain Of One Peyote Remix,1
｢H×M×-Crash-Trap@｣ －A Type－,1


In [15]:
df = orig_df[~orig_df['track-name'].isna()]
df.isna().sum()

user_id                        0
timestamp                      0
musicbrainz-artist-id     519381
artist-name                    0
musicbrainz-track-id     1878081
track-name                     0
dtype: int64

In [16]:
df

,user_id,timestamp,musicbrainz-artist-id,artist-name,musicbrainz-track-id,track-name
0,user_000001,2009-05-04T23:08:57Z,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Deep Dish,NaN,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007
1,user_000001,2009-05-04T13:54:10Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Composition 0919 (Live_2009_4_15)
2,user_000001,2009-05-04T13:52:04Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc2 (Live_2009_4_15)
3,user_000001,2009-05-04T13:42:52Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Hibari (Live_2009_4_15)
4,user_000001,2009-05-04T13:42:11Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc1 (Live_2009_4_15)
...,...,...,...,...,...,...
15960385,user_000829,2008-10-25T02:40:43Z,55704c38-224f-4b75-b29f-d43653f8bc9a,Primal Scream,b8975af5-8877-471b-a352-cac079d6e760,Rocks
15960386,user_000829,2008-10-25T02:38:44Z,55704c38-224f-4b75-b29f-d43653f8bc9a,Primal Scream,fa26f72c-a81b-41a1-8db2-673e5b1578df,Jailbird
15960387,user_000829,2008-10-25T02:36:04Z,381e8434-109a-439c-8acc-5d328359f2e0,The Brian Jonestown Massacre,85c71fd5-9ec7-4e8f-ab06-11709a973362,All Around You (Outro)
15960388,user_000829,2008-10-25T02:32:19Z,381e8434-109a-439c-8acc-5d328359f2e0,The Brian Jonestown Massacre,e18180ec-57b0-4257-b247-6c0d7f3594b5,Here It Comes


# generate an track_id

In [18]:
data_clean = df[['timestamp','user_id', 'artist-name','track-name']]
data_clean = data_clean.set_index(['artist-name' ,'track-name'])
data_clean

timestamp  \
artist-name                  track-name                                                         
Deep Dish                    Fuck Me Im Famous (Pacha Ibiza)-09-28-2007  2009-05-04T23:08:57Z   
坂本龍一                         Composition 0919 (Live_2009_4_15)           2009-05-04T13:54:10Z   
                             Mc2 (Live_2009_4_15)                        2009-05-04T13:52:04Z   
                             Hibari (Live_2009_4_15)                     2009-05-04T13:42:52Z   
                             Mc1 (Live_2009_4_15)                        2009-05-04T13:42:11Z   
...                                                                                       ...   
Primal Scream                Rocks                                       2008-10-25T02:40:43Z   
                             Jailbird                                    2008-10-25T02:38:44Z   
The Brian Jonestown Massacre All Around You (Outro)                      2008-10-25T02:36:04Z   
                             Here It Comes                               2008-10-25T02:32:19Z   
                             Slowdown (Fuck Tomorrow)                    2008-10-25T02:31:19Z   

                                                                             user_id  
artist-name                  track-name                                               
Deep Dish                    Fuck Me Im Famous (Pacha Ibiza)-09-28-2007  user_000001  
坂本龍一                         Composition 0919 (Live_2009_4_15)           user_000001  
                             Mc2 (Live_2009_4_15)                        user_000001  
                             Hibari (Live_2009_4_15)                     user_000001  
                             Mc1 (Live_2009_4_15)                        user_000001  
...                                                                              ...  
Primal Scream                Rocks                                       user_000829  
                             Jailbird                                    user_000829  
The Brian Jonestown Massacre All Around You (Outro)                      user_000829  
                             Here It Comes                               user_000829  
                             Slowdown (Fuck Tomorrow)                    user_000829  

[15960378 rows x 2 columns]

In [29]:
data_clean.sort_index(inplace=True)
mi = data_clean.index.drop_duplicates()
mi[:5]

MultiIndex([(                    '! Europe - France - Cold Wave', ...),
            ('! Germany - Surfbeat From Beyond The Iron Curtain', ...),
            (                   '! Www.Polskie-Mp3.Tk ! Adam Mi', ...),
            (                  '! Www.Polskie-Mp3.Tk ! Breakout', ...),
            (                  '! Www.Polskie-Mp3.Tk ! Breakout', ...)],
           names=['artist-name', 'track-name'])

In [22]:
len(mi)

1340450

In [33]:
data_clean.loc[mi[500]]

timestamp      user_id
artist-name         track-name                                        
'T Hof Van Commerce Niemand Grodder  2008-09-22T19:01:58Z  user_000316
                    Niemand Grodder  2008-02-06T19:22:25Z  user_000316

In [38]:
data_clean['track_id'] = 'track_x'
for i, m in enumerate(mi):
    data_clean.loc[m, 'track_id'] = 'track_'+str(i)

KeyboardInterrupt: 